## 4. Kerchunk input data and the kerchunk API

Within this series, we cannot explain how kerchunking works. For now, it is only important to understand that it leverages the zarr benefits of both small memory requirements for opening as well as consolidated metadata for virtual aggregation.

We now design the script such that it 
- opens *kerchunk* references instead of files
- enables access trough the kerchunk API

With the kerchunk API, we do not necessarily need a dask cluster anymore (but without a dask cluster, the dask API will not work).

**Differences to the first example**:

- we open data through the so called *lazy reference* mapper with
    ```python
    fsspec.get_mapper(
        lazy=True,
        )
    ```
    which we pass to xarray afterwards. This only works for kerchunked input data.
- we add a *dict* of fspec mappern to the kerchunk plguin by setting `kp.mapper_dict`
    Xpublish will recognize the

In [1]:
%%writefile xpublish_references.py

port=9000
ssl_keyfile="/work/bm0021/k204210/cloudify/workshop/key.pem"
ssl_certfile="/work/bm0021/k204210/cloudify/workshop/cert.pem"

from cloudify.plugins.stacer import *
from cloudify.utils.daskhelper import *
from cloudify.plugins.kerchunk import *
import xarray as xr
import xpublish as xp
import asyncio
import nest_asyncio
import sys
import os

nest_asyncio.apply()

if __name__ == "__main__":  # This avoids infinite subprocess creation
    #import dask
    #zarrcluster = asyncio.get_event_loop().run_until_complete(get_dask_cluster())
    #os.environ["ZARR_ADDRESS"]=zarrcluster.scheduler._address
    
    dsname=sys.argv[1]
    glob_inp=sys.argv[2]

    dsdict={}
    source="reference::/"+glob_inp
    fsmap = fsspec.get_mapper(
        source,
        remote_protocol="file",
        lazy=True,
        cache_size=0
    )
    ds=xr.open_dataset(
        fsmap,
        engine="zarr",
        chunks="auto",
        consolidated=False        
    )
    kp = KerchunkPass()
    kp.mapper_dict = {source:fsmap}
    ds=ds.drop_encoding()    
    ds.encoding["source"]=source
    dsdict[dsname]=ds
    
    collection = xp.Rest(dsdict)
    collection.register_plugin(Stac())
    collection.register_plugin(kp)
    collection.serve(
        host="0.0.0.0",
        port=port,
        ssl_keyfile=ssl_keyfile,
        ssl_certfile=ssl_certfile
    )

Overwriting xpublish_references.py


We run this app with ERA5 data:

```
dsname="era5"
glob_inp="/work/bm1344/DKRZ/kerchunks_single/testera/E5_sf_an_1D.parquet"
```

by applying:

In [2]:
%%bash --bg
source activate /work/bm0021/conda-envs/cloudify
python xpublish_references.py era5 /work/bm1344/DKRZ/kerchunks_single/testera/E5_sf_an_1D.parquet

If sth goes wrong, you can check for *cloudify* processes that you can *kill* by ID.

In [21]:
!ps -ef | grep cloudify

k204210  1753538 1752638  0 08:07 ?        00:00:26 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-16868865-4ac9-4f10-9d8e-59c1bb69e85a.json
k204210  1753566 1752638  0 08:07 ?        00:00:18 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-4fe110eb-67f5-471d-843b-d47538c216e2.json
k204210  1761151 1753566  0 15:57 pts/2    00:00:00 /bin/bash -c ps -ef | grep cloudify
k204210  1761153 1761151  0 15:57 pts/2    00:00:00 grep cloudify


In [14]:
!kill 1539362

**Data access via the kerchunk API**

You can get the host url with the hostname of the levante node you work on and the port that you used for the app:

In [13]:
port=9000
hostname=!echo $HOSTNAME
hosturl="https://"+hostname[0]+":"+str(port)
print(hosturl)

https://l40354.lvt.dkrz.de:9000


We have to tell the python programs to do not verify ssl certificates for our purposes:

In [14]:
storage_options=dict(verify_ssl=False)

**Xarray**

Our era dataset is available via both the *zarr* API **and** the *kerchunk* API.
They are named similar:

In [17]:
dsname="era5"
zarr_url='/'.join([hosturl,"datasets",dsname,"zarr"])
kerchunk_url='/'.join([hosturl,"datasets",dsname,"kerchunk"])
print(kerchunk_url)

https://l40354.lvt.dkrz.de:9000/datasets/era5/kerchunk


In [18]:
import xarray as xr
ds=xr.open_zarr(
    kerchunk_url,
    consolidated=True,
    storage_options=storage_options
)

In [19]:
ds

<xarray.Dataset> Size: 5TB
Dimensions:  (time: 30955, cell: 542080)
Coordinates:
    lat      (cell) float64 4MB dask.array<chunksize=(542080,), meta=np.ndarray>
    lon      (cell) float64 4MB dask.array<chunksize=(542080,), meta=np.ndarray>
  * time     (time) datetime64[ns] 248kB 1940-01-01T11:30:00 ... 2024-09-30T1...
Dimensions without coordinates: cell
Data variables: (12/40)
    100u     (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    100v     (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    10u      (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    10v      (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    2d       (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    2t       (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    ...       ...
    swvl4    (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    tcc      (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    tco3     (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    tcw      (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    tcwv     (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    tsn      (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
Attributes: (12/22)
    project:              ECMWF Re-Analysis
    project_id:           ERA
    institution_id:       ECMWF-DKRZ
    institution:          Data from European Centre for Medium-Range Weather ...
    source_id:            IFS
    source:               ECMWF Integrated Forecast System (IFS) CY41R2
    ...                   ...
    format:               kerchunk
    product:              reanalysis
    responsible_persons:  Angelika Heil, Fabian Wachsmann
    title:                The DKRZ ERA5 data pool. Generated using Copernicus...
    license:              The ERA5 data are published with the Copernicus Pro...
    references:           Hersbach, H., Bell, B., Berrisford, P., Hirahara, S...

In [50]:
ds.isel(time=-1).load()

<xarray.Dataset> Size: 182MB
Dimensions:  (cell: 542080)
Coordinates:
    lat      (cell) float64 4MB 89.78 89.78 89.78 89.78 ... -89.78 -89.78 -89.78
    lon      (cell) float64 4MB nan 20.0 40.0 60.0 ... 280.0 300.0 320.0 340.0
    time     datetime64[ns] 8B 2024-09-30T11:30:00
Dimensions without coordinates: cell
Data variables: (12/40)
    100u     (cell) float64 4MB 5.838 2.636 -0.8782 ... -8.296 -6.063 -2.922
    100v     (cell) float64 4MB -8.215 -9.827 -10.22 ... -6.401 -8.995 -10.81
    10u      (cell) float64 4MB 4.36 2.381 0.09148 ... -5.739 -4.711 -3.002
    10v      (cell) float64 4MB -4.953 -6.245 -6.762 ... -3.089 -4.959 -6.483
    2d       (cell) float64 4MB 259.5 259.7 259.9 260.1 ... 219.4 219.4 219.4
    2t       (cell) float64 4MB 261.9 262.1 262.3 262.5 ... 223.3 223.4 223.5
    ...       ...
    swvl4    (cell) float64 4MB 0.0 0.0 0.0 0.0 ... 0.1602 0.1602 0.1602 0.1602
    tcc      (cell) float64 4MB 0.9721 0.9683 0.9667 ... 0.9813 0.9838 0.9871
    tco3     (cell) float64 4MB 0.006883 0.006877 0.006873 ... 0.002953 0.002952
    tcw      (cell) float64 4MB 5.733 5.772 5.815 5.877 ... 0.4126 0.4126 0.4126
    tcwv     (cell) float64 4MB 5.655 5.692 5.733 5.793 ... 0.3989 0.4008 0.4008
    tsn      (cell) float64 4MB 261.7 261.9 262.1 262.4 ... 221.2 221.3 221.3
Attributes: (12/22)
    project:              ECMWF Re-Analysis
    project_id:           ERA
    institution_id:       ECMWF-DKRZ
    institution:          Data from European Centre for Medium-Range Weather ...
    source_id:            IFS
    source:               ECMWF Integrated Forecast System (IFS) CY41R2
    ...                   ...
    format:               kerchunk
    product:              reanalysis
    responsible_persons:  Angelika Heil, Fabian Wachsmann
    title:                The DKRZ ERA5 data pool. Generated using Copernicus...
    license:              The ERA5 data are published with the Copernicus Pro...
    references:           Hersbach, H., Bell, B., Berrisford, P., Hirahara, S...

**Intake**

The default **method** for intake datasets is *kerchunk* i.e. the datasets are loaded through the kerchunk API per default.

In [20]:
intake_url='/'.join([hosturl,"intake.yaml"])
print(intake_url)

https://l40354.lvt.dkrz.de:9000/intake.yaml


In [53]:
import intake
cat=intake.open_catalog(
    intake_url,
    storage_options=storage_options
)
list(cat)

['era5']

In [54]:
cat[dsname](storage_options=storage_options).to_dask()

/work/bm0021/conda-envs/cloudify/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


<xarray.Dataset> Size: 5TB
Dimensions:  (time: 30955, cell: 542080)
Coordinates:
    lat      (cell) float64 4MB dask.array<chunksize=(542080,), meta=np.ndarray>
    lon      (cell) float64 4MB dask.array<chunksize=(542080,), meta=np.ndarray>
  * time     (time) datetime64[ns] 248kB 1940-01-01T11:30:00 ... 2024-09-30T1...
Dimensions without coordinates: cell
Data variables: (12/40)
    100u     (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    100v     (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    10u      (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    10v      (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    2d       (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    2t       (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    ...       ...
    swvl4    (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    tcc      (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    tco3     (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    tcw      (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    tcwv     (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
    tsn      (time, cell) float64 134GB dask.array<chunksize=(1, 542080), meta=np.ndarray>
Attributes: (12/22)
    project:              ECMWF Re-Analysis
    project_id:           ERA
    institution_id:       ECMWF-DKRZ
    institution:          Data from European Centre for Medium-Range Weather ...
    source_id:            IFS
    source:               ECMWF Integrated Forecast System (IFS) CY41R2
    ...                   ...
    format:               kerchunk
    product:              reanalysis
    responsible_persons:  Angelika Heil, Fabian Wachsmann
    title:                The DKRZ ERA5 data pool. Generated using Copernicus...
    license:              The ERA5 data are published with the Copernicus Pro...
    references:           Hersbach, H., Bell, B., Berrisford, P., Hirahara, S...

In [22]:
stac_url=zarr_url.replace('/zarr','/stac')

In [26]:
import pystac
import fsspec
import json
pystac.item.Item.from_dict(
    json.load(fsspec.open(stac_url,**storage_options).open())
)

<Item id=era5>

In [ ]:
!cdo=/work/bm0021/cdo_incl_cmor/cdo-test_cmortest_gcc/bin/cdo